# Find Intro in a Video

In [1]:
from skimage.measure import compare_ssim
from sklearn.metrics.pairwise import cosine_similarity
import argparse
import imutils
import cv2
import os
import pandas as pd

In [2]:
def load_dataset(path_dataset):
    
    datasets = []
    
    #listar arquivos do dataset de abertura
    paths = os.listdir(path_dataset)
    paths.sort()
    for ds_file in paths:
        
        if not ds_file.endswith('01.jpg'):
            continue

        # criar path do arquivo
        file_ds_frame_path = "{}/{}".format(path_dataset, ds_file)
        print(file_ds_frame_path)
        
        # ler a imagem
        img_file = cv2.imread(file_ds_frame_path)
        
        # converter para cinza
        gray_file = cv2.cvtColor(img_file, cv2.COLOR_BGR2GRAY)
        
        mean, eigenVectors = cv2.PCACompute(gray_file, mean=None, maxComponents=10)

        eigenVectors = eigenVectors[:10]
        # adiciona a imagem na lista
        datasets.append({'grayfile': gray_file, 'eigenvectors': eigenVectors, 'filename': ds_file})
    
    return datasets 

In [4]:
PATH_CSV_CENAS = 'scenes_tmp/segundo_sol_bloco_1_completo-Scenes.csv'
PATH_CENAS_VIDEO = "scenes_tmp/segundo_sol_bloco_1_completo"
PATH_DATASET_ABERTURA = "scenes_tmp/segundo_sol"

In [5]:
df_metadados_cenas = pd.read_csv(PATH_CSV_CENAS, skiprows=1)
dataset_abertura = load_dataset(PATH_DATASET_ABERTURA)
frames_video = load_dataset(PATH_CENAS_VIDEO)

scenes_tmp/segundo_sol/segundo_sol-Scene-001-01.jpg
scenes_tmp/segundo_sol/segundo_sol-Scene-002-01.jpg
scenes_tmp/segundo_sol/segundo_sol-Scene-003-01.jpg
scenes_tmp/segundo_sol/segundo_sol-Scene-004-01.jpg
scenes_tmp/segundo_sol/segundo_sol-Scene-005-01.jpg
scenes_tmp/segundo_sol/segundo_sol-Scene-006-01.jpg
scenes_tmp/segundo_sol/segundo_sol-Scene-007-01.jpg
scenes_tmp/segundo_sol/segundo_sol-Scene-008-01.jpg
scenes_tmp/segundo_sol/segundo_sol-Scene-009-01.jpg
scenes_tmp/segundo_sol/segundo_sol-Scene-010-01.jpg
scenes_tmp/segundo_sol/segundo_sol-Scene-011-01.jpg
scenes_tmp/segundo_sol/segundo_sol-Scene-012-01.jpg
scenes_tmp/segundo_sol/segundo_sol-Scene-013-01.jpg
scenes_tmp/segundo_sol/segundo_sol-Scene-014-01.jpg
scenes_tmp/segundo_sol/segundo_sol-Scene-015-01.jpg
scenes_tmp/segundo_sol/segundo_sol-Scene-016-01.jpg
scenes_tmp/segundo_sol_bloco_1_completo/segundo_sol_bloco_1_completo-Scene-001-01.jpg
scenes_tmp/segundo_sol_bloco_1_completo/segundo_sol_bloco_1_completo-Scene-002-01.

In [6]:
SIMLARITY_THRESHOLD = 0.8

from sklearn.metrics.pairwise import cosine_similarity

def calcular_similaridade_via_eigenvectors(frame1, frame2):
    score = cosine_similarity(frame1['eigenvectors'], frame2['eigenvectors'])
    print('{} vs {}:   {}'.format(frame1['filename'], frame2['filename'], score[0][0]))
    return score[0][0] >= SIMLARITY_THRESHOLD


def calcular_similaridade_via_grayscale(frame1, frame2):
    (score, _) = compare_ssim(frame1['grayfile'], frame2['grayfile'], full=True)
    print('{} vs {}:   {}'.format(frame1['filename'], frame2['filename'], score))
    return score >= SIMLARITY_THRESHOLD

FN_COMPARACAO = calcular_similaridade_via_eigenvectors

def cena_é_parte_da_abertura(frame_cena, datasets_abertura):
    for frame_abr in datasets_abertura:
        if FN_COMPARACAO(frame_cena, frame_abr):
            return True
    return False

def encontrar_cenas_abertura(df_metadados_cenas, dataset_abertura, frames_video):
    lista_cenas = []
    
#     for index, row in df_metadados_cenas.iterrows():
    for frame_video in frames_video:
#         print("Processando cena {}".format(row["Scene Number"]))
#         scene_number = row["Scene Number"]
        
        if cena_é_parte_da_abertura(frame_video, dataset_abertura):
            print("A cena faz parte da abertura !")
            lista_cenas.append(frame_video)
        else:
            print("A cena não faz parte !")
    return lista_cenas


cenas_abertura = encontrar_cenas_abertura(df_metadados_cenas, dataset_abertura, frames_video)
print(cenas_abertura)


segundo_sol_bloco_1_completo-Scene-001-01.jpg vs segundo_sol-Scene-001-01.jpg:   -0.6728996634483337
segundo_sol_bloco_1_completo-Scene-001-01.jpg vs segundo_sol-Scene-002-01.jpg:   0.6125936508178711
segundo_sol_bloco_1_completo-Scene-001-01.jpg vs segundo_sol-Scene-003-01.jpg:   -0.8228126168251038
segundo_sol_bloco_1_completo-Scene-001-01.jpg vs segundo_sol-Scene-004-01.jpg:   0.6382135152816772
segundo_sol_bloco_1_completo-Scene-001-01.jpg vs segundo_sol-Scene-005-01.jpg:   -0.05835118889808655
segundo_sol_bloco_1_completo-Scene-001-01.jpg vs segundo_sol-Scene-006-01.jpg:   0.6860597729682922
segundo_sol_bloco_1_completo-Scene-001-01.jpg vs segundo_sol-Scene-007-01.jpg:   -0.6924397349357605
segundo_sol_bloco_1_completo-Scene-001-01.jpg vs segundo_sol-Scene-008-01.jpg:   -0.3465789258480072
segundo_sol_bloco_1_completo-Scene-001-01.jpg vs segundo_sol-Scene-009-01.jpg:   0.6658783555030823
segundo_sol_bloco_1_completo-Scene-001-01.jpg vs segundo_sol-Scene-010-01.jpg:   -0.3632079064